## 1. 파일 불러오기

### 1-1) csv파일 불러오기

#### 판례본문

In [2]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel

# CSV 파일에서 클러스터링 결과 로드
df_inheritance = pd.read_csv('df_inheritance_keyword.csv')

c:\Users\user\.conda\envs\nof\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 판례목록

In [4]:
# 사건 리스트 로드
inheritance_list = pd.read_csv('inheritance_list.csv')

# 사건명 추가를 위해 df_inheritance 데이터프레임과 병합
df_inheritance = df_inheritance.merge(inheritance_list[['사건번호', '사건명']], on='사건번호', how='left')

### 1-2) MYSQL데이터 불러오기

In [1]:
!pip install mysql-connector-python

   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   ----------------------- ---------------- 9.4/16.1 MB 97.6 MB/s eta 0:00:01
   -------------------------- ------------- 10.5/16.1 MB 26.2 MB/s eta 0:00:01
   ---------------------------------------- 16.1/16.1 MB 28.1 MB/s eta 0:00:00


In [ ]:
#py파일불러오기
import sys
import os

sys.path.append(r'C:\Users\82102\Downloads\no-f\NoF')

import db_info

#### 판례본문

In [ ]:
import mysql.connector
import pandas as pd
from db_info import user,password,host,database,port

db_connection = mysql.connector.connect(
    host = host,
    user = user,
    password = password,
    database = database,
    port = port
)

cursor = db_connection.cursor()

cursor.execute("SELECT * FROM inheritance_keyword")

result = cursor.fetchall()

inheritance_content = pd.DataFrame(result, columns = [i[0] for i in cursor.description])

inheritance_content

#### 판례목록

In [ ]:
import mysql.connector
import pandas as pd
from db_info import user,password,host,database,port

db_connection = mysql.connector.connect(
    host = host,
    user = user,
    password = password,
    database = database,
    port = port
)

cursor = db_connection.cursor()

cursor.execute("SELECT * FROM inheritance_list")

result = cursor.fetchall()

inheritance_content = pd.DataFrame(result, columns = [i[0] for i in cursor.description])

inheritance_content

### 2) 데이터확인

In [5]:
df_inheritance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4166 entries, 0 to 4165
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   판례정보일련번호       4166 non-null   int64 
 1   사건번호           4166 non-null   object
 2   선고             4166 non-null   object
 3   판결유형           4166 non-null   object
 4   판시사항           3250 non-null   object
 5   판결요지           2932 non-null   object
 6   참조조문           3225 non-null   object
 7   참조판례           1935 non-null   object
 8   판례내용           4166 non-null   object
 9   판례내용_상단        4166 non-null   object
 10  판례내용_이유        4166 non-null   object
 11  판례내용_이유_전처리    4146 non-null   object
 12  판례내용_이유_불용어제거  4131 non-null   object
 13  클러스터           4166 non-null   int64 
 14  키워드            4166 non-null   object
 15  사건명            3924 non-null   object
dtypes: int64(2), object(14)
memory usage: 520.9+ KB


## 2. 임베딩 후 CSV로 저장

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# KOBERT 모델과 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert')

# 문장 임베딩 함수
def encode_sentence(sentence):
    inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

# 판례 내용, 키워드, 판결요지, 판시사항 임베딩 생성
df_inheritance['판례내용이유임베딩'] = df_inheritance['판례내용_이유'].apply(lambda x: encode_sentence(x).tolist() if pd.notnull(x) else None)
df_inheritance['키워드임베딩'] = df_inheritance['키워드'].apply(lambda x: encode_sentence(x).tolist() if pd.notnull(x) else None)
df_inheritance['판결요지임베딩'] = df_inheritance['판결요지'].apply(lambda x: encode_sentence(x).tolist() if pd.notnull(x) else None)
df_inheritance['판시사항임베딩'] = df_inheritance['판시사항'].apply(lambda x: encode_sentence(x).tolist() if pd.notnull(x) else None)

# CSV 파일로 저장
df_inheritance.to_csv('df_inheritance_kobertmodel.csv', index=False)

#### ** 생성데이터 확인

In [8]:
df_inheritance.head(2)

,판례정보일련번호,사건번호,선고,판결유형,판시사항,판결요지,참조조문,참조판례,판례내용,판례내용_상단,판례내용_이유,판례내용_이유_전처리,판례내용_이유_불용어제거,클러스터,키워드,사건명,판례내용이유임베딩,키워드임베딩,판결요지임베딩,판시사항임베딩
0,238913,2023다221144,선고,판결,"[1] 가분채권이 공동상속되는 경우의 법률관계 / 주식이 공동상속되는 경우, 공동...",[1] 금전채권과 같이 급부의 내용이 가분인 채권은 공동상속되는 경우 상속개시와 ...,"[1] 민법 제269조, 제278조, 제547조 제1항, 제1005조, 제1006...","[1] 대법원 2003. 5. 30. 선고 2003다7074 판결, 대법원 200...","【원고, 상고인】 원고 (소송대리인 법무법인 참진 담당변호사 이영동)【피고, 피상고...","【원고, 상고인】 원고 (소송대리인 법무법인 참진 담당변호사 이영동)【피고, 피상고...",【이 유】 상고이유를 판단한다. 1. 사안의 개요 가. 사실관계 원...,상고이유를 판단한다. 1. 사안의 개요 가. 사실관계 원심판결 이유와 기록에 따르면...,개요 사실관계 망인 은행 예금 채권 은행 판매 투자신탁 형태 MMF Money Ma...,2,"[신청, 상속, 부동산, 주택, 사망, 건물, 계약, 회사, 재산, 호주]",예금[투자신탁 형태 단기금융집합투자기구(MMF) 수익권의 공동상속 효과가 문제된 사건],"[[0.09682950377464294, -0.2833014726638794, 0....","[[0.08193794637918472, 0.05260368064045906, 0....","[[0.12045687437057495, -0.09459716826677322, 0...","[[0.20212049782276154, -0.06857649981975555, 0..."
1,240673,2021두52143,선고,판결,대한민국에서 주로 체류하며 대한민국에 있는 건축사무소 등에서 건축설계사로 근무하...,NaN,구 섭외사법(2001. 4. 7. 법률 제6465호 국제사법으로 전부 개정되기 전의...,NaN,"【원고, 상고인】 원고 1 외 3인 (소송대리인 변호사 이종건)【피고, 피상고인】 ...","【원고, 상고인】 원고 1 외 3인 (소송대리인 변호사 이종건)【피고, 피상고인】 ...","【이 유】 상고이유를 판단한다. 1. 원심판결 이유와 기록에 따르면, 다...","상고이유를 판단한다. 1. 원심판결 이유와 기록에 따르면, 다음의 사실을 알 수 있...",인과 대한민국 혼인신고 마쳤고 자녀 미합중국 캘리포니아 출생 미합중국 시민권 취...,2,"[신청, 상속, 부동산, 주택, 사망, 건물, 계약, 회사, 재산, 호주]",상속세부과처분취소,"[[0.1609489917755127, -0.2994731664657593, 0.6...","[[0.08193794637918472, 0.05260368064045906, 0....",None,"[[0.053899895399808884, 0.21528321504592896, 0..."


## 3. 문장임베딩함수 활용하는 유사도 계산 코드

#### ** transformers실행되는 환경에서만 사용가능

#### 1) 판결요지

In [11]:
# 사용자 입력
user_keyword = "토지"  # 사용자가 입력한 키워드
user_keyword_embedding = encode_sentence(user_keyword)

# 판례내용 임베딩 유사도 계산 및 사건명 추출
content_similarity_scores = []

for idx, row in df_inheritance.iterrows():
    if row['판결요지임베딩'] is not None:

        content_similarity = cosine_similarity(user_keyword_embedding, row['판결요지임베딩']).flatten()[0]
        content_similarity_scores.append((row['사건번호'], content_similarity))

# 유사도 기준으로 정렬 및 상위 N개 선택
sorted_content_scores = sorted(content_similarity_scores, key=lambda x: x[1], reverse=True)
top_n_contents = sorted_content_scores[:10]

# 추천 사건명 출력
print("사용자 키워드와 관련된 사건명:")
for 사건번호, score in top_n_contents:
    사건명 = inheritance_list.loc[inheritance_list['사건번호'] == 사건번호, '사건명'].values
    사건명 = 사건명[0] if 사건명.size > 0 else "사건명 없음"
    print(f"사건번호: {사건번호}, 사건명: {사건명}, 유사도: {score:.4f}")

사용자 키워드와 관련된 사건명:
사건번호: 4294민상168, 사건명: 덕대계약이행완료확인, 유사도: 0.5380
사건번호: 81다카550, 사건명: 소유권이전등기말소, 유사도: 0.5114
사건번호: 86다카1312, 사건명: 소유권이전등기말소등, 유사도: 0.4960
사건번호: 4294민상1255, 사건명: 가옥명도, 유사도: 0.4863
사건번호: 62다343, 사건명: 부동산소유권이전등기말소, 유사도: 0.4846
사건번호: 87다카663, 사건명: 소유권이전등기말소등, 유사도: 0.4839
사건번호: 90다카17214, 사건명: 소유권이전등기말소, 유사도: 0.4827
사건번호: 80구796, 사건명: 양도소득세부과처분취소, 유사도: 0.4824
사건번호: 80구844, 사건명: 취득세부과처분취소, 유사도: 0.4824
사건번호: 90다카3789, 사건명: 소유권이전등기말소등, 유사도: 0.4777


#### 2) 판례내용_이유

In [12]:
# 사용자 입력
user_keyword = "토지"  # 사용자가 입력한 키워드
user_keyword_embedding = encode_sentence(user_keyword)

# 판례내용 임베딩 유사도 계산 및 사건명 추출
content_similarity_scores = []

for idx, row in df_inheritance.iterrows():
    if row['판례내용이유임베딩'] is not None:
        # 판례내용 임베딩과 사용자 키워드 임베딩 간의 유사도 계산
        content_similarity = cosine_similarity(user_keyword_embedding, row['판례내용이유임베딩']).flatten()[0]
        content_similarity_scores.append((row['사건번호'], content_similarity))

# 유사도 기준으로 정렬 및 상위 N개 선택
sorted_content_scores = sorted(content_similarity_scores, key=lambda x: x[1], reverse=True)
top_n_contents = sorted_content_scores[:10]

# 추천 사건명 출력
print("사용자 키워드와 관련된 사건명 :")
for 사건번호, score in top_n_contents:
    사건명 = df_inheritance.loc[df_inheritance['사건번호'] == 사건번호, '사건명'].values
    사건명 = 사건명[0] if 사건명.size > 0 else "사건명 없음"
    print(f"사건번호: {사건번호}, 사건명: {사건명}, 유사도: {score:.4f}")

사용자 키워드와 관련된 사건명 :
사건번호: 2010누30330, 사건명: 상속세 부과처분 취소, 유사도: 0.3974
사건번호: 87누898, 사건명: 증여세등부과처분취소, 유사도: 0.3690
사건번호: 2000으2, 사건명: 상속재산관리인선임, 유사도: 0.3683
사건번호: 83누221, 사건명: 상속세부과처분취소, 유사도: 0.3666
사건번호: 2012누8801, 사건명: 상속세부과처분취소, 유사도: 0.3662
사건번호: 2009브23, 사건명: 상속한정승인결정에대한즉시항고, 유사도: 0.3646
사건번호: 80다2360, 사건명: 소유권이전등기말소, 유사도: 0.3645
사건번호: 79다1009, 사건명: 소유권보존등기말소, 유사도: 0.3628
사건번호: 84누649, 사건명: 상속세부과처분취소, 유사도: 0.3625
사건번호: 2006다2179, 사건명: 상속분양수, 유사도: 0.3621


#### 3) 판시사항

In [13]:
# 사용자 입력
user_keyword = "토지"  # 사용자가 입력한 키워드
user_keyword_embedding = encode_sentence(user_keyword)

# 판례내용 임베딩 유사도 계산 및 사건명 추출
content_similarity_scores = []

for idx, row in df_inheritance.iterrows():
    if row['판시사항임베딩'] is not None:
        # 판례내용 임베딩과 사용자 키워드 임베딩 간의 유사도 계산
        content_similarity = cosine_similarity(user_keyword_embedding, row['판시사항임베딩']).flatten()[0]
        content_similarity_scores.append((row['사건번호'], content_similarity))

# 유사도 기준으로 정렬 및 상위 N개 선택
sorted_content_scores = sorted(content_similarity_scores, key=lambda x: x[1], reverse=True)
top_n_contents = sorted_content_scores[:10]

# 추천 사건명 출력
print("사용자 키워드와 관련된 사건명 :")
for 사건번호, score in top_n_contents:
    사건명 = inheritance_list.loc[inheritance_list['사건번호'] == 사건번호, '사건명'].values
    사건명 = 사건명[0] if 사건명.size > 0 else "사건명 없음"
    print(f"사건번호: {사건번호}, 사건명: {사건명}, 유사도: {score:.4f}")

사용자 키워드와 관련된 사건명 :
사건번호: 95누9327, 사건명: 상속세부과처분취소, 유사도: 0.8622
사건번호: 2007다9719, 사건명: 소유권이전등기, 유사도: 0.8622
사건번호: 85누804, 사건명: 증여세부가처분취소, 유사도: 0.8622
사건번호: 83다카893, 사건명: 부당이득금, 유사도: 0.8622
사건번호: 83다카745, 사건명: 대여금, 유사도: 0.8622
사건번호: 82누138, 사건명: 양도소득세부과처분취소, 유사도: 0.8622
사건번호: 81나459, 사건명: 공유물분할청구사건, 유사도: 0.8622
사건번호: 80다2360, 사건명: 소유권이전등기말소, 유사도: 0.8622
사건번호: 80나4240, 사건명: 손해배상청구사건, 유사도: 0.8622
사건번호: 79다434, 사건명: 소유권보존등기말소, 유사도: 0.8622
